In [1]:
import wandb
from collections import deque
import torch
import numpy as np


In [2]:
try:
    key = key or input()
except NameError:
    key = input('type your wandb api key: ')

In [3]:
wandb.login(key=key)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: felissi. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\felixwong/.netrc


True

In [4]:
wandb.init(project="my-awesome-project")

Exception in thread Exception in thread NetStatThr:
ChkStopThr:
Traceback (most recent call last):
  File "c:\Users\felixwong\Anaconda3\lib\threading.py", line 973, in _bootstrap_inner
Traceback (most recent call last):
  File "c:\Users\felixwong\Anaconda3\lib\threading.py", line 973, in _bootstrap_inner
    self.run()
  File "c:\Users\felixwong\Anaconda3\lib\threading.py", line 910, in run
    self.run()
  File "c:\Users\felixwong\Anaconda3\lib\threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Users\felixwong\Anaconda3\lib\site-packages\wandb\sdk\wandb_run.py", line 256, in check_network_status
    self._target(*self._args, **self._kwargs)
  File "c:\Users\felixwong\Anaconda3\lib\site-packages\wandb\sdk\wandb_run.py", line 274, in check_stop_status
    self._loop_check_status(    
  File "c:\Users\felixwong\Anaconda3\lib\site-packages\wandb\sdk\wandb_run.py", line 214, in _loop_check_status
self._loop_check_status(
  File "c:\Users\felixwong\Anac

Problem at: C:\Users\felixwong\AppData\Local\Temp\ipykernel_9404\3596492707.py 1 <cell line: 1>


wandb: ERROR Abnormal program exit


: 

: 

In [3]:
from Agent import Agent
from Board import Board

In [19]:
BUFFER_SIZE         = int(1e4)
BATCH_SIZE          = 64
GAMMA               = 0.90  # discount factor
TAU                 = 1e-3  # soft update of target parameter
LEARNING_RATE       = 1e-3
UPDATE_EVERY        = 10    # how often to update the local
TARGET_UPDATE_EVERY = 50    # how often to update the target
STATE_SIZE          = 16
ACTION_SIZE         = 4   

In [20]:
agent = Agent(STATE_SIZE, ACTION_SIZE, LEARNING_RATE, BUFFER_SIZE, BATCH_SIZE)
env = Board()


In [6]:
wandb.watch(agent.qnetwork_local,log_freq=10)

ValueError: You must call `wandb.init` before calling watch

## Monitoring metrics and parameters

### Metrics for how the model doing and updating
1. q values
1. gradients
1. loss
1. eps
1. 


### Metrics for model's performance
1. largest number of the board
1. accumulate reward
1. number of rounds

In [21]:
def train(agent: Agent, n_episodes, max_time_step, eps_start, eps_end, eps_decay):
    scores = []
    num_rounds = []
    scores_window = deque(maxlen=100)
    eps = eps_start
    for episode in range(n_episodes):
        env.reset()
        state = env.board
        accumulate_reward = 0
        rounds = 0
        for time_step in range(max_time_step):
            action_values = agent.q_value(state, eps)
            action = agent.decide(action_values, eps)
            next_state, reward, done = env.step(action)
            agent.step(state, action, reward, next_state, done)
            """ === this step has finished === """
            wandb.log({'action':action, 'reward': reward, 'eps': eps})
            wandb.log({f'action_values[{i}]':q for i, q in enumerate(action_values.cpu().numpy().flatten()) })
            if agent.time_step % UPDATE_EVERY == 0 and len(agent.memory) > BATCH_SIZE:
                loss = agent.learn_from_experience()
                wandb.log({'loss':loss})
            if agent.time_step % TARGET_UPDATE_EVERY == 0:
                agent.soft_update()
            """ === next iteration === """
            state = next_state
            accumulate_reward += reward
            rounds += 1
            if done:
                wandb.log({'rounds':rounds,'accumulate_reward':accumulate_reward, 'max_number':np.max(state)})
                break
        scores_window.append(accumulate_reward)
        scores.append(accumulate_reward)
        num_rounds.append(rounds)
        eps = max(eps_end, eps-eps_decay)
        if episode % 100 == 0:
            print(episode, np.mean(scores_window))
            torch.save(agent.qnetwork_local.state_dict(), 'checkpoint.pt')
    return scores, num_rounds

In [22]:
train(agent, n_episodes=1000, max_time_step=1000, eps_start=1.0, eps_end=0.05, eps_decay=0.01)

{'rounds': 75, 'accumulate_reward': 1460, 'max_number': 32, 'loss': tensor(609.5389, grad_fn=<MseLossBackward0>)}
0 1460.0
{'rounds': 57, 'accumulate_reward': 1060, 'max_number': 32, 'loss': tensor(542.6468, grad_fn=<MseLossBackward0>)}
{'rounds': 168, 'accumulate_reward': 2180, 'max_number': 128, 'loss': tensor(417.6039, grad_fn=<MseLossBackward0>)}
{'rounds': 145, 'accumulate_reward': 1970, 'max_number': 128, 'loss': tensor(332.1172, grad_fn=<MseLossBackward0>)}
{'rounds': 126, 'accumulate_reward': 1770, 'max_number': 128, 'loss': tensor(370.2298, grad_fn=<MseLossBackward0>)}
{'rounds': 92, 'accumulate_reward': 1430, 'max_number': 64, 'loss': tensor(255.6561, grad_fn=<MseLossBackward0>)}
{'rounds': 80, 'accumulate_reward': 1500, 'max_number': 32, 'loss': tensor(338.3432, grad_fn=<MseLossBackward0>)}
{'rounds': 115, 'accumulate_reward': 1890, 'max_number': 64, 'loss': tensor(471.4345, grad_fn=<MseLossBackward0>)}
{'rounds': 95, 'accumulate_reward': 1410, 'max_number': 64, 'loss': tens

([1460,
  1060,
  2180,
  1970,
  1770,
  1430,
  1500,
  1890,
  1410,
  2020,
  2020,
  2030,
  1900,
  2090,
  2040,
  1960,
  1600,
  1890,
  2150,
  1610,
  2070,
  1490,
  1880,
  1560,
  1720,
  2000,
  1840,
  1500,
  1220,
  1550,
  1740,
  1530,
  2020,
  1590,
  1450,
  1810,
  1680,
  1790,
  1290,
  1940,
  1990,
  1360,
  2050,
  1540,
  1510,
  1910,
  2420,
  1900,
  1630,
  2150,
  1680,
  2160,
  1640,
  1790,
  1350,
  1650,
  1700,
  1880,
  1650,
  2300,
  2120,
  2190,
  1760,
  1730,
  1910,
  1950,
  1250,
  1840,
  2310,
  1530,
  1900,
  1580,
  1530,
  2210,
  2040,
  2190,
  1200,
  1900,
  1370,
  1550,
  1740,
  1640,
  1360,
  2320,
  1430,
  1250,
  1670,
  1600,
  1670,
  1340,
  1720,
  1670,
  1620,
  1940,
  1770,
  1860,
  2290,
  1170,
  1690,
  1730,
  1700,
  1500,
  2020,
  1960,
  2080,
  2030,
  1980,
  1390,
  1870,
  1490,
  2120,
  1790,
  1390,
  1890,
  1490,
  1820,
  1650,
  1860,
  1100,
  1470,
  1280,
  1520,
  1780,
  1770,
  1780,


In [23]:
agent.q_value(np.array([[0,0,2,0],[0,0,2,0],[0,0,2,0],[0,0,2,0]])).cpu().numpy().flatten()

array([58.216633, 56.67393 , 55.787964, 56.341194], dtype=float32)

In [24]:
agent.act(np.array([[0,0,2,0],[0,0,2,0],[0,0,2,0],[0,0,2,0]]), eps=0)

0